# Yet another example of Agency.

In [ ]:
import os
import textwrap
import uuid

import shapely
from rich.pretty import pprint

from gait import Agency, Agent, ObserverLoguru, OperatorLiteLLM, Scratchpad

In [ ]:
suffix = "gpt-4o-preview"
model = f"azure/{suffix}"
api_base = f"{os.environ['AZURE_API_URL']}/{suffix}"

In [ ]:
def buffer_geometry(
        geom_uuid: str,
        distance_in_meters: float,
        scratchpad: Scratchpad,
) -> str:
    """Buffer a geometry referenced by a UUID by a distance in meters.
    Return the buffered geometry UUID.

    :param geom_uuid: UUID of a geometry to buffer.
    :param distance_in_meters: The buffer distance in meters.
    :param scratchpad: Scratchpad instance injected at runtime.
    """
    geom = scratchpad[geom_uuid]
    buff = shapely.buffer(geom, distance_in_meters)
    buff_uuid = str(uuid.uuid4())
    scratchpad[buff_uuid] = buff
    return buff_uuid

In [ ]:
buffer_agent = Agent(
    model=model,
    name="BufferAgent",
    description="Buffer a geometry UUID by a distance",
    instructions="Buffer a geometry referenced by a UUID by a distance in meters.",
    functions=[buffer_geometry],
    params=dict(
        api_base=api_base,
        temperature=0.0,
        tool_choice="required",
    ),
)

In [ ]:
def wkt_to_geometry(
        wkt: str,
        scratchpad: Scratchpad,
) -> str:
    """Parse WKT string into a geometry UUID.
    The following are WKT string samples:
    POINT (30 10)
    LINESTRING (30 10, 10 30, 40 40)
    POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))

    :param wkt: The well known text to convert to a geometry UUID.
    :param scratchpad:  Scratchpad instance injected at runtime.
    """
    geom = shapely.from_wkt(wkt, on_invalid="ignore")
    geom_uuid = str(uuid.uuid4())
    scratchpad[geom_uuid] = geom
    return geom_uuid

In [ ]:
wkt_agent = Agent(
    model=model,
    name="WKTAgent",
    description=textwrap.dedent(
        """
    Convert WKT string to geometry UUID.
    
    The following are WKT string samples:
    POINT (30 10)
    LINESTRING (30 10, 10 30, 40 40)
    POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))    
    """
    ),
    functions=[wkt_to_geometry],
    params=dict(
        api_base=api_base,
        temperature=0.0,
        tool_choice="required",
    ),
)

In [ ]:
report_agent = Agent(
    model=model,
    name="ReportAgent",
    description="""Use this agent to report the final answer. Make sure to suffix the final answer with <ANSWER/>""",
    params=dict(
        api_base=api_base,
        temperature=0.2,
        stop=["<ANSWER/>"],
    ),
)

In [ ]:
operator = OperatorLiteLLM(
    model,
    agents=[report_agent, buffer_agent, wkt_agent],
    params=dict(
        api_base=api_base,
        tool_choice="required",
    ),
)

In [ ]:
agency = Agency(operator, observer=ObserverLoguru())

for _ in agency("Get the UUID of the buffer of WKT POINT(10 5) by 10 meters."):
    if _.agent == report_agent:
        agency.terminate()
        print(_.content)

### Get the messages in the dialog.

In [ ]:
for _ in agency.dialog:
    pprint(_, expand_all=True)

### Get the scratchpad content.

In [ ]:
for _ in agency.scratchpad:
    pprint(_, expand_all=True)